In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  roc_auc_score
from sklearn.datasets import make_moons
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  roc_auc_score
from sklearn.datasets import make_moons
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from custom_addons.lead_scoring_ai.data.preprocessing_finalized import file_path





In [19]:

# Step 1: Load Data
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(df.info())
    return df

# Step 2: Select Columns
def select_columns(df, columns):
    selected_df = df[columns]
    print(selected_df.head())
    return selected_df

# Step 3: Handle Missing Values
def fill_missing_country(df):
    # Extract unique City-Country mapping
    city_country_map = df.dropna(subset=['City', 'Country']).set_index('City')['Country'].to_dict()
    # Fill missing 'Country' values based on 'City'
    df['Country'] = df.apply(lambda row: city_country_map.get(row['City'], row['Country']), axis=1)
    print(city_country_map)
    return df

def handle_missing_specialization(df):
    # Fill missing values in 'Specialization'
    df['Specialization'] = df['Specialization'].fillna('Select')
    return df

# Step 4: Generate New Columns
def add_decision_authority(df):
    authority_levels = [5, 4, 3, 2, 1]
    probabilities = [0.05, 0.15, 0.25, 0.35, 0.20]
    df['Decision_Authority'] = np.random.choice(authority_levels, size=len(df), p=probabilities)
    return df

def add_job_position(df):
    job_titles_map = {
        5: ['CEO', 'CFO', 'CTO', 'COO', 'Chief Innovation Officer'],
        4: ['Vice President', 'Director of Operations', 'Head of Marketing', 'Senior Strategy Director'],
        3: ['Project Manager', 'Marketing Manager', 'Team Lead', 'Operations Manager'],
        2: ['Data Analyst', 'Software Engineer', 'Business Consultant', 'Technical Specialist'],
        1: ['Intern', 'Junior Developer', 'Assistant Coordinator', 'Associate Analyst']
    }
    df['Job Position'] = df['Decision_Authority'].apply(lambda x: np.random.choice(job_titles_map.get(x, ['Unknown'])))
    return df


def extract_decision_authority(df, job_titles_map):
    """
    Reads the 'Job Position' column and extracts the corresponding Decision Authority level
    based on the provided job_titles_map, creating a new column.

    Args:
    df (pd.DataFrame): The DataFrame containing the 'Job Position' column.
    job_titles_map (dict): A dictionary mapping Decision Authority levels to job titles.

    Returns:
    pd.DataFrame: The DataFrame with a new 'Extracted Decision Authority' column.
    """
    # Reverse the mapping: Job Title → Decision Authority
    reversed_map = {}
    for authority, titles in job_titles_map.items():
        for title in titles:
            reversed_map[title] = authority

    # Extract Decision Authority using the reversed mapping
    df['Extracted Decision Authority'] = df['Job Position'].map(reversed_map).fillna(0)  # Fill unknown titles with 0 or any default

    return df



# Step 5: Encode Binary Columns
def encode_binary_columns(df, columns, encoding_rules):
    for col in columns:
        encoded_col = f"{col} Encoded"
        df[encoded_col] = df[col].str.lower().map(encoding_rules).fillna(0)
    df.drop(columns, axis=1, inplace=True)
    return df

# Step 6: One-Hot Encoding
def one_hot_encode(df, columns):
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded_data = encoder.fit_transform(df[columns])
    joblib.dump(encoder, 'fitted_encoder.joblib')

    # Set the index of the encoded DataFrame to match df's index
    encoded_df = pd.DataFrame(
        encoded_data,
        columns=encoder.get_feature_names_out(columns),
        index=df.index  # This is the key change
    )

    df = pd.concat([df.drop(columns, axis=1), encoded_df], axis=1)
    return df


Splitting Data

Pipeline


Considering that all the models have similar outputs. Random forests seem to be an optimal choice since they capture non-linearity and are less computationally expensive than gradient boosting. Logistic regression yielded good scores but, in the future, if the dataset is more complex, it will probably fail to capture its patterns.

We will further fine-tune and calibrate it.

In [9]:
# # first accuracy:  0.9128
# first f1: 0.88


#  2nd accuracy: 0.9134 , n_estimators=200
#  2nd f1: 0.89

# 3rd accuracy:0.9139  , n_estimators=500
# 3rd f1: 0.89

# # according to available litrature, the obtained f1 score is considered good.
#
# from sklearn.ensemble import RandomForestClassifier
#
# # Initialize and train
# rf = RandomForestClassifier(n_estimators=500, random_state=42)
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
#
# # Metrics
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))
# # Get raw probabilities (uncalibrated)

Accuracy: 0.913961038961039
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1136
           1       0.90      0.88      0.89       712

    accuracy                           0.91      1848
   macro avg       0.91      0.91      0.91      1848
weighted avg       0.91      0.91      0.91      1848



Calibrating Random Forests

Problem: RFs tend to produce overconfident probabilities (e.g., predicting 0.95 when true likelihood is 0.7)

Solution: Use Platt Scaling (logistic calibration) or Isotonic Regression to align probabilities with actual outcomes

In [10]:
# from sklearn.calibration import CalibratedClassifierCV
#
# # Calibrate with Platt Scaling (sigmoid)
# calibrated_rf = CalibratedClassifierCV(
#     estimator=rf,
#     method='sigmoid',  # or 'isotonic' for non-parametric
#     cv=5,             # Use 5-fold cross-validation
#     ensemble=True
# )
#
# calibrated_rf.fit(X_train, y_train)
#


CalibratedClassifierCV(cv=5, ensemble=True,
                       estimator=RandomForestClassifier(n_estimators=500,
                                                        random_state=42))

In [11]:
# Get raw probabilities (uncalibrated)
# y_prob_uncalibrated = rf.predict_proba(X_test)[:, 1]
#
# print(y_prob_uncalibrated)
#
# # 4. Add probabilities back to DataFrame (for analysis)
# test_df = X_test.copy()
# test_df['uncalibrated_prob'] = y_prob_uncalibrated
# test_df['actual'] = y_test  # Optional: Compare with ground truth
#
# #print(test_df[['uncalibrated_prob', 'actual']].head(20))
#
#
# # Get predicted probabilities
# probab = calibrated_rf.predict_proba(X_test)
#
# # Extract the probability of the positive class (e.g., lead becoming a sale)
# y_prob_calibrated = probab[:, 1]
#
# test_df['rf_calibrated_prob'] = y_prob_calibrated
#
# print(test_df[['rf_calibrated_prob', 'uncalibrated_prob', 'actual']].head(20))
#


[0.13606667 0.95866667 0.14133571 ... 0.98984762 0.02       0.16701735]
      rf_calibrated_prob  uncalibrated_prob  actual
683             0.109643           0.136067       1
1931            0.936771           0.958667       1
6950            0.100824           0.141336       0
2996            0.039090           0.002000       0
3902            0.042838           0.007000       0
6828            0.063773           0.058000       0
4906            0.955106           0.998273       0
5796            0.401640           0.433305       0
6388            0.087223           0.140970       0
3636            0.043046           0.018369       0
3685            0.957941           0.998000       1
3279            0.052489           0.058742       0
3799            0.124784           0.097917       0
9158            0.851825           0.830367       1
5766            0.037231           0.000000       0
4341            0.950784           0.974000       1
3043            0.047784           0.042051 

In [12]:
# import joblib
#
# joblib.dump(calibrated_rf, 'calibrated_rf_odoo.joblib')

['calibrated_rf_odoo.joblib']

In [22]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import OneHotEncoder


# Make sure that the following helper functions are defined/imported in your file:
# - fill_missing_country(df)
# - handle_missing_specialization(df)
# - add_job_position(df)
# - extract_decision_authority(df, job_titles_map)
# - encode_binary_columns(df, binary_columns, encoding_rules)
# - one_hot_encode(df, categorical_columns)

class JobPositionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, mode="train"):
        # mode can be "train" or "inference"
        self.mode = mode
        self.job_titles_map = {
            5: ['CEO', 'CFO', 'CTO', 'COO', 'Chief Innovation Officer'],
            4: ['Vice President', 'Director of Operations', 'Head of Marketing', 'Senior Strategy Director'],
            3: ['Project Manager', 'Marketing Manager', 'Team Lead', 'Operations Manager'],
            2: ['Data Analyst', 'Software Engineer', 'Business Consultant', 'Technical Specialist'],
            1: ['Intern', 'Junior Developer', 'Assistant Coordinator', 'Associate Analyst']
        }

    def fit(self, X, y=None):
        # No fitting needed in this transformer
        return self

    def transform(self, X):
        X = X.copy()
        # In training mode, generate the Job Position column
        if self.mode == "train":
            # Assumes add_job_position is defined externally and applies your business logic.
            X = add_decision_authority(X)
            X = add_job_position(X)
        # For both training and inference, extract decision authority
        X = extract_decision_authority(X, self.job_titles_map)
        # Drop the 'Job Position' column if it exists
        if 'Job Position' in X.columns:
            X.drop('Job Position', axis=1, inplace=True)
        return X


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

class LeadPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Define your rules and lists
        self.encoding_rules = {'yes': 1, 'y': 1, 'no': 0, 'n': 0}
        self.binary_columns = ['Through Recommendations', 'Do Not Call', 'Do Not Email']
        self.categorical_columns = ['Lead Source', 'Country', 'Tags', 'Specialization', 'City', 'Last Notable Activity']
        # List of identifier columns to drop.
        self.identifier_columns = ['id', 'Prospect ID', 'Lead Number']  # update as needed

        # Initialize the OneHotEncoder with handle_unknown='ignore'
        self.one_hot_enc = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

    def fit(self, X, y=None):
        # Make a copy and drop unwanted identifier columns
        df = X.copy()
        df = df.drop([col for col in self.identifier_columns if col in df.columns], axis=1)

        # You could also apply your fill_missing_country and handle_missing_specialization here if needed:
        df = fill_missing_country(df)
        df = handle_missing_specialization(df)
        df = encode_binary_columns(df, self.binary_columns, self.encoding_rules)

        # Fit the one-hot encoder on categorical columns
        self.one_hot_enc.fit(df[self.categorical_columns])
        return self

    def transform(self, X):
        df = X.copy()
        # Drop identifier columns
        df = df.drop([col for col in self.identifier_columns if col in df.columns], axis=1)

        # Apply your preprocessing functions
        df = fill_missing_country(df)
        df = handle_missing_specialization(df)
        df = encode_binary_columns(df, self.binary_columns, self.encoding_rules)

        # For one-hot encoding, use the already fitted encoder
        encoded_data = self.one_hot_enc.transform(df[self.categorical_columns])
        encoded_df = pd.DataFrame(
            encoded_data,
            columns=self.one_hot_enc.get_feature_names_out(self.categorical_columns),
            index=df.index
        )
        # Drop original categorical columns and concatenate the encoded dataframe
        df = pd.concat([df.drop(self.categorical_columns, axis=1), encoded_df], axis=1)
        return df


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('job_position_processing', JobPositionTransformer(mode="train")),  # Keep your custom transformer here
    ('preprocessing', LeadPreprocessor()),
    ('classifier', RandomForestClassifier(n_estimators=500, random_state=42))
])

# Example usage:
# Assume that you have loaded your full dataset into a DataFrame `df`
# and that the target column in your dataset is called "Target".
file_path = r'C:\Program Files\Odoo 18.0.20250413\custom_addons\lead_scoring_ai\data\Lead Scoring.csv'
df = pd.read_csv(file_path)
# Split your data into features and target.

selected_columns= ['Lead Source','Do Not Email', 'Do Not Call', 'Country', 'City','Specialization', 'Through Recommendations' , 'Last Notable Activity','Tags']


X = df[selected_columns]
y = df['Converted']

# Further split into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Columns in X_train:", X_train.columns.tolist())



# # Train (fit) the pipeline using your training data.
pipeline.fit(X_train, y_train)
#
# # Evaluate the model on test data.
score = pipeline.score(X_test, y_test)
print("Test Score:", score)

# # Save the fully trained pipeline to disk for later inference.
joblib.dump(pipeline, 'trained_pipeline.joblib')

Columns in X_train: ['Lead Source', 'Do Not Email', 'Do Not Call', 'Country', 'City', 'Specialization', 'Through Recommendations', 'Last Notable Activity', 'Tags']
{'Thane & Outskirts': 'India', 'Select': 'India', 'Tier II Cities': 'India', 'Mumbai': 'India', 'Other Cities of Maharashtra': 'India', 'Other Cities': 'India', 'Other Metro Cities': 'India'}
{'Thane & Outskirts': 'India', 'Select': 'India', 'Tier II Cities': 'India', 'Mumbai': 'India', 'Other Cities of Maharashtra': 'India', 'Other Cities': 'India', 'Other Metro Cities': 'India'}
{'Mumbai': 'India', 'Other Cities': 'India', 'Select': 'India', 'Thane & Outskirts': 'India', 'Other Cities of Maharashtra': 'India', 'Tier II Cities': 'India', 'Other Metro Cities': 'India'}
Test Score: 0.9161255411255411


C:\Users\islam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


['trained_pipeline.joblib']

In [ ]:
# import joblib
#
# # Load the saved pipeline
# inference_pipeline = joblib.load('trained_pipeline.joblib')
#
# # Update the mode of JobPositionTransformer to 'inference'
# # This tells the transformer not to generate (or alter) job positions, but just to extract decision authority.
# inference_pipeline.named_steps['job_position_processing'].mode = "inference"
#
# # Now, you can directly call predict on your new raw data.
# predictions = inference_pipeline.predict(new_data)  # new_data must contain all expected columns

# print(predictions)
